# Homework 3: EDA of Bike Sharing

## Course Policies

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** at the top
of your solution.

## Due Date

** This assignment is due Tuesday, February 13 at 11:59pm **

## Introduction

After completing this assignment, you should be comfortable with:

* reading plaintext delimited data into `pandas`
* wrangling data for analysis
* using EDA to learn about your data 
* making informative plots

This assignment includes both specific tasks to perform and open-ended questions to investigate. The open-ended questions ask you to think creatively and critically about how the plots you have created provide insight into the data.

There are four parts to this assignment: 
* data preparation
* exploring the distribution of riders
* exploring the relationship between time and rider counts
* and exploring the relationship between weather and rider counts. 

In each section, you are given specific data cleaning tasks and/or a plot to reproduce.  

In [ ]:
# Run this cell to set up your notebook.  Make sure utils.py is in this assignment's folder

import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from pathlib import Path
import utils

# Default plot configurations
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,8)
plt.rcParams['figure.dpi'] = 150
sns.set()

from IPython.display import display, Latex, Markdown


## Loading Bike Sharing Data
The data we are exploring is data on bike sharing in Washington D.C.

The variables in this data frame are defined as:

Variable       | Description
-------------- | ------------------------------------------------------------------
instant | record index
dteday | date
season | 1. spring <br> 2. summer <br> 3. fall <br> 4. winter
yr | year (0: 2011, 1:2012)
mnth | month ( 1 to 12)
hr | hour (0 to 23)
holiday | whether day is holiday or not
weekday | day of the week
workingday | if day is neither weekend nor holiday
weathersit | 1. clear or partly cloudy <br> 2. mist and clouds <br> 3. light snow or rain <br> 4. heavy rain or snow
temp | normalized temperature in Celsius (divided by 41)
atemp | normalized "feels-like" temperature in Celsius (divided by 50)
hum | normalized percent humidity (divided by 100)
windspeed| normalized wind speed (divided by 67)
casual | count of casual users
registered | count of registered users
cnt | count of total rental bikes including casual and registered  

### Download the Data

In [ ]:
# Run this cell to download the data.  No further action is needed

data_url = 'https://github.com/DS-100/sp18/raw/gh-pages/assets/datasets/hw3-bikeshare.zip'
file_name = 'data.zip'
data_dir = '.'

dest_path = utils.fetch_and_cache(data_url=data_url, data_dir=data_dir, file=file_name)
print('Saved at {}'.format(dest_path))

zipped_data = zipfile.ZipFile(dest_path, 'r')

data_dir = Path('data')
zipped_data.extractall(data_dir)


print("Extracted Files:")
for f in data_dir.glob("*"):
    print("\t",f)

### Examining the file contents

Can you identify the file format? (No answer required.)

In [ ]:
# Run this cell to look at the top of the file.  No further action is needed
for line in utils.head(data_dir/'bikeshare.txt'):
    print(line,end="")

### Size
Is the file big?  How many records do we expect to find? (No answers required.)

In [ ]:
# Run this cell to view some metadata.  No further action is needed
print("Size:", (data_dir/"bikeshare.txt").stat().st_size, "bytes")
print("Line Count:", utils.line_count(data_dir/"bikeshare.txt"), "lines")

### Loading the data

The following code loads the data into a Pandas DataFrame.

In [ ]:
# Run this cell to load the data.  No further action is needed
bike = pd.read_csv(data_dir/'bikeshare.txt')
bike.head()

## 1. Data Preparation
A few of the variables that are numeric/integer actually encode categorical data. These include `holiday`, `weekday`, `workingday`, and `weathersit`. In the following problem, we will convert these four variables to strings specifying the categories. In particular, use 3-letter labels (`Sun`, `Mon`, `Tue`, `Wed`, `Thu`, `Fri`, and `Sat`) for `weekday`. You may simply use `yes`/`no` for `holiday` and `workingday`. 

In this exercise we will *mutate* the data frame, **overwriting the corresponding variables in the data frame.** However, be sure to document the data transformation and leave the underlying datafile `bikeshare.txt` unmodified.

#### Question 1a (Decoding `weekday`, `workingday`, and `weathersit`)


Decode the `holiday`, `weekday`, `workingday`, and `weathersit` fields:

1. `holiday`: Convert to `yes` and `no`.  Hint: There are fewer holidays...
1. `weekday`: It turns out that Monday is the day with the most holidays.  Mutate the `'weekday'` column to use the 3-letter label (`'Sun'`, `'Mon'`, `'Tue'`, `'Wed'`, `'Thu'`, `'Fri'`, and `'Sat'` ...) instead of its current numerical values.
1. `workingday`: Convert to `yes` and `no`.
1. `weathersit`: You should replace each value with one of `Clear`, `Mist`, `Light`, or `Heavy`.

In [ ]:
# Modify holiday weekday, workingday, and weathersit here
# Hint: one strategy involves df.replace(...)

### BEGIN SOLUTION
factor_dict = {
    'holiday': {
        0:'no', 
        1:'yes'
    },
    'weekday': {
        0: 'Sun', 
        1: 'Mon', 
        2: 'Tue', 
        3: 'Wed', 
        4: 'Thu', 
        5: 'Fri', 
        6: 'Sat'
    },
    'workingday': {
        0: 'no',
        1: 'yes'
    },
    'weathersit': {
        1: 'Clear',
        2: 'Mist',
        3: 'Light',
        4: 'Heavy'
    }
}

bike.replace(factor_dict, inplace=True)
bike.head()
### END SOLUTION

In [ ]:
assert isinstance(bike, pd.DataFrame)
assert bike['holiday'].dtype == np.dtype('O')
assert list(bike['holiday'].iloc[370:375]) == ['no', 'no', 'yes', 'yes', 'yes']
assert bike['weekday'].dtype == np.dtype('O')
assert bike['workingday'].dtype == np.dtype('O')
assert bike['weathersit'].dtype == np.dtype('O')

### BEGIN HIDDEN TESTS
assert bike.shape == (17379, 17) or bike.shape == (17379, 18)
assert list(bike['weekday'].iloc[::2000]) == ['Sat', 'Tue', 'Mon', 'Mon', 'Mon', 'Sun', 'Sun', 'Sat', 'Sun']
assert list(bike['workingday'].iloc[::2000]) == ['no', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no']
assert list(bike['weathersit'].iloc[::2000]) == ['Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear', 'Clear']
### END HIDDEN TESTS

#### Question 1b (Holidays)

How many entries in the data correspond to holidays?  Set the variable `num_holidays` to this value.

In [ ]:
num_holidays = ...

### BEGIN SOLUTION
num_holidays = bike['holiday'].value_counts()['yes']
### END SOLUTION

In [ ]:
assert 400 <= num_holidays <= 550

### BEGIN HIDDEN TESTS
assert num_holidays == 500
### END HIDDEN TESTS

#### Question 1c (Computing Daily Total Counts)

The granularity of this data is at the hourly level.  However, for some of the analysis we will also want to compute daily statistics.  In particular, in the next few questions we will be analyzing the daily number of registered and unregistered users.

Construct a data frame with the following columns:
* `casual`: total number of casual riders for each day
* `registered`: total number of registered riders for each day
* `workingday`: whether that day is a working day or not (`yes` or `no`)

Hint: `groupby` and `aggregate`.  `aggregate` admits the `sum` and `'last'` aggregation operators. 

In [ ]:
daily_counts = ...

### BEGIN SOLUTION
daily_counts = (
    bike
    .groupby(['dteday'])
    .aggregate(
        {
        "casual": sum, 
        "registered": sum, 
        "workingday": 'last'
        }
    )
)
daily_counts.head()
### END SOLUTION

In [ ]:
assert np.round(daily_counts['casual'].mean()) == 848.0
assert np.round(daily_counts['casual'].var()) == 471450.0

### BEGIN HIDDEN TESTS
assert np.round(daily_counts['registered'].mean()) == 3656.0
assert np.round(daily_counts['registered'].var()) == 2434400.0
assert sorted(list(daily_counts['workingday'].value_counts())) == [231, 500]
### END HIDDEN TESTS

## 2. Exploring the Distribution of Riders

Let's begin by comparing the distribution of the daily counts of casual and registered riders.  

### Question 2a

Use the [`sns.distplot`](https://seaborn.pydata.org/generated/seaborn.distplot.html) function to create a plot that overlays the distribution of the daily counts of `casual` and `registered` users. The temporal granularity of the records should be daily counts, which you should have after completing question 1c.

You may want to quickly review the [seaborn plotting tutorial](https://seaborn.pydata.org/tutorial/distributions.html).

**Hint:** *Don't forget to set the `plt.xlabel` as well as the labels for each distribution.*

In [ ]:
...

### BEGIN SOLUTION
sns.distplot(daily_counts['casual'], label='casual')
sns.distplot(daily_counts['registered'],  label='registered', color='green')
plt.legend()
plt.xlabel("Rider Count")
### END SOLUTION

### Question 2b

Describe the differences you notice between the density curves for casual and registered riders.  Consider concepts such as modes, symmetry, skewness, tails, gaps and outliers.  Include a comment on the spread of the distributions.

The casual riders distribution has a sharp peak at 1000 that may be bimodal. 
This distribution is skewed right and has a long right tail with a small set of daily counts around 2500.

The distribution of registered riders has a more symmetric distribution centered around almost 4000 daily riders.
This distribution does not have heavy skew. It's spread is much wider than the casual riders. 

### Question 2c

The density plots do not show us how the daily counts for registered and casual riders vary together. Use [`sns.lmplot`](https://seaborn.pydata.org/generated/seaborn.lmplot.html) to make a scatter plot to investigate the relationship between casual and registered counts. Color the points in the scatterplot according to whether or not the day is working day. There are many points in the scatter plot so make them small to help with over plotting.

* **Hint 1:** *Checkout this helpful [tutorial on `lmplot`](https://seaborn.pydata.org/tutorial/regression.html)*.

* **Hint 2:** *You will need to set `x`, `y`, and `hue` and the `scatter_kws`*.

In [ ]:
# Make the font size a bit bigger
sns.set(font_scale=1.5)

...

### BEGIN SOLUTION
sns.set(font_scale=1.5)
sns.lmplot(x="casual", y="registered", hue="workingday",
           data=bike, fit_reg=False, size=8, scatter_kws={"s": 10})
### END SOLUTION

### Question 2d

What does this scatterplot seem to reveal about the relationship (if any) between casual and registered riders and whether or not the day is on the weekend?
Why might we be concerned with overplotting in examining this relationship? 


There appears to be a linear relationship between the counts for registered and casual riders,
and this relationship depends on whether the day is a work day or a weekend day.

Given the overplotting, it is difficult to discern the relationships because
for example, we can't see if there might be blue points that have been 
plotted over by green points and vice versa.

## Question 3

### Question 3a Bivariate Kernel Density Plot
 
The scatter plot you made in question 2c makes clear the separation between the work days and non-work days.  However, the overplotting
makes it difficult to see the density of the joint counts. To address this
issue, let's try visualizing the data with another technique, the kernel density plot.

You will want to read up on the documentation for `sns.kdeplot` which can be found at https://seaborn.pydata.org/generated/seaborn.kdeplot.html

The result we wish to achieve should be a plot that looks like this:

<img src='images/bivariate_kde_of_daily_rider_types.png' width="600px" />

Making this plot can be complicated so we will provide a walkthrough below, feel free to use whatever method you wish however if you do not want to follow the walkthrough.

* **Hint 1:** *You will need to make more than one KDE plot.*
* **Hint 2:** *You will want to set the `cmap` to `"Reds"` and `"Blues"` or at least two contrasting colors. *

In [ ]:
# Set 'ind' to a boolean condition that will allow you to filter daily_counts for rows corresponding to workingdays
ind = ...

# Bivariate KDEs require two data inputs. 
# In this case, we will need the daily counts for casual and registered riders on weekdays
# Hint: use loc and ind to splice out the relevant rows and column (casual/registered)
casual_weekday = ...
registered_weekday = ...

# Use sns.kdeplot to plot the bivariate KDE for weekday rides

# Repeat the same steps above but for rows corresponding to non-workingdays
casual_weekend = ...
registered_weekend = ...

### BEGIN SOLUTION
ind = daily_counts['workingday'] == "yes"

sns.kdeplot(daily_counts.loc[ind, "casual"], daily_counts.loc[ind, "registered"], 
            cmap="Reds")
sns.kdeplot(daily_counts.loc[~ind, "casual"], daily_counts.loc[~ind, "registered"],
            cmap="Blues")
plt.savefig("images/bivariate_kde_of_daily_rider_types.png", bbox_inches='tight', dpi=300)
### END SOLUTION

### Question 3b

What does the contour plot reveal about the relationship between casual and registered riders for both work days and non-work days? How is it an improvement over the scatter plot you created in Q2c?

On the non-workdays, the casual and registered riders have a roughly linear relationship. 
During the work week, the relationship also appears roughly linear,
but the slope is much higher. 

Here we can see the shape of the relationship because high density regions 
are revealed that we were not able to see due to overplotting in the scatter plot.

### Question 3c

As an alternative approach, construct the following set of three plots where the main plot shows the contours of the kernel density estimate of daily counts for registered and casual riders and the two "margin" plots provide the univariate kernel density estimate of each of these variables. 

<img src="images/joint_distribution_of_daily_rider_types.png" width="600px" />

Hints:
* Take a look at `sns.jointplot` and its `kind` parameter. 
* `set_axis_labels` can be used to rename axes

In [ ]:
...

### BEGIN SOLUTION
sns.jointplot(x="casual", y="registered", data=daily_counts,  kind="kde")\
   .set_axis_labels('Daily Count Casual Riders', 'Daily Count Registered Riders')

plt.savefig("images/joint_distribution_of_daily_rider_types.png", bbox_inches='tight', dpi=300)
### END SOLUTION

## 4. Exploring Ride Sharing and Time

### Question 4a

Plot number of riders for each hour of each day in the month of June in 2011. 

Make sure to add descriptive x-axis and y-axis labels and create a legend to distinguish the line for casual riders and the line for registered riders. The end result should look like this:

<img src="images/june_riders.png" width="600px" />

In [ ]:
...

### BEGIN SOLUTION

june = bike[(bike["mnth"]==6) & (bike["yr"]==0)].copy()
june['day'] = (june['instant'] - june['instant'].min())/24.0
june = june.set_index("day")

# plt.figure(figsize=(10, 7))
june[["casual", "registered"]].plot()
plt.legend(loc=2)
plt.ylabel("Number of Riders")
plt.xlabel("Day of the Month")

## Uncomment this to save new plot for question.
plt.savefig("images/june_riders.png", bbox_inches='tight', dpi=300)

### END SOLUTION

#### Question 4b

This plot has several interesting features. How do the number of casual and registered riders compare for different days of the month? What is an interesting trend and pattern you notice between the lines?

We can see several things in this line plot. We see daily patterns in rental. 
With the weekends having far fewer registered users and more casual riders. 
The number of casual riders is always less than the number of registerred riders, 
but on the week end, the counts are much closer together. 

## 5. Understanding Daily Patterns

### Question 5a
Let's examine the behavior of riders by plotting the average number of riders for each hour of the day over the entire dataset, stratified by rider type.  

Your plot should look like the following:

<img src="images/diurnal_bikes.png" width="600px"/>

In [ ]:
...

### BEGIN SOLUTION
plt.figure(figsize=(8, 5))
data = bike.groupby("hr")[["casual","registered"]].mean()
data[["casual", "registered"]].plot()
plt.legend(loc=2)
plt.xlabel("Hour of the Day")
plt.ylabel("Average Count")

## Uncomment this to render the instruction picture
plt.savefig("images/diurnal_bikes.png", bbox_inches='tight', dpi=300)
### END SOLUTION

### Question 5b

What can you observe from the plot?  Hypothesize about the meaning of the peaks in the registered riders' distribution.

In the above plot we see strong evidence of daily patterns in both datasets.  
The casual riders appear to ride throughout the day with peak hours in the mid-afternoon. 
Alternatively, while the registered riders also ride more during the day than at night there are
very strong spikes during the morning and evening commute hours with a small bump during lunch.  

## 6. Exploring Ride Sharing and Weather
Now let's examine how the weather is affecting rider's behavior. First let's look at how the proportion of casual riders changes as weather changes.

### Question 6a
Create a new column `prop_casual` in the `bike` dataframe representing the proportion of casual riders out of all riders.

In [ ]:
...

### BEGIN SOLUTION
bike["prop_casual"] = bike["casual"] / bike["cnt"]
### END SOLUTION

In [ ]:
assert int(bike["prop_casual"].sum()) == 2991

### BEGIN HIDDEN TEST
assert np.round(bike["prop_casual"].mean(), 2) == 0.17
### END HIDDEN TEST

### Question 6b
In order to examine the relationship between proportion of casual riders and temerature, we can make some bivariate scatterplot using `sns.lmplot`. We can even use color/hue to encode the information about day of week. Run the following cells to create such plots. (The plot contains many points, so it may take a while to render)

In [ ]:
sns.lmplot(data=bike, x="temp", y="prop_casual", hue="weekday", scatter_kws={"s": 20}, size=10)

As you can see from the scatterplot, many points are overlapping. Though the plot can show some trends, it would be hard to verify your findings. Therefore we could fit some curves to summarize the data and plot the curves instead.

Basically you will need to make a plot like this: Fit and plot curves using data from different weekdays. 

<img src="images/curveplot_temp_prop_casual.png" width="600px" />

You will need to use the function [`statsmodels.nonparametric.smoothers_lowess.lowess`](http://www.statsmodels.org/dev/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html) to fit a curve from two sequences `x` and `y`. An example on a random data set is given below.



In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
# Make noisy data
xobs = np.sort(np.random.rand(100)*4.0 - 2)
yobs = np.exp(xobs) + np.random.randn(100) / 2.0
plt.plot(xobs, yobs, 'o', label="Raw Data")

# Predict 'smoothed' valued for observations
ysmooth = lowess(yobs, xobs, return_sorted=False)
plt.plot(xobs, ysmooth, 'r-', label="Smoothed Estimator")
plt.legend()

In [ ]:
# Reproduce the lowess plot for the bike data here
...

### BEGIN SOLUTION
from statsmodels.nonparametric.smoothers_lowess import lowess
days = ['Sun','Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
for day in days:
    _ = bike[bike["weekday"] == day].copy()
    _["temp"] = _["temp"] * 41
    lowess_est = lowess(_["prop_casual"], _["temp"] * 41, return_sorted=False)    
    plt.plot(_["temp"], lowess_est, label=day)
plt.xlabel("Temperature (Celcius)")
plt.ylabel("Casual Rider Proportion")
plt.legend()
plt.savefig("images/curveplot_temp_prop_casual", bbox_inches='tight', dpi=300)
### END SOLUTION

### Question 6c
What can you discover from the curve plot? How is the `prop_casual` changing as a function of temperature?

1. As temperature increase, the proportion of casual riders increase as well.
1. Weekends (Saturday, Sunday) have higher proportion of casual riders

# Submission

You're done!

In order to turn in this assignment, submit this notebook to the Data 100 datahub at http://data100.datahub.berkeley.edu. 

You will need to upload this notebook and any associated files to datahub manually if you have completed this assignment on your local machine. Detailed instructions for how to submit on Datahub can be found at http://www.ds100.org/sp18/materials.

Remember to click 'Validate' for this assignment before submitting. After clicking 'Submit', you can verify there is a time-stamped submission under 'Submitted assignments'.